# COGS 118B - Final Project

# Names

- Sam Dimmock
- Vincent Gao
- Owen Guan
- Ryota Takemura


# Abstract 
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize: 
- what your goal/problem is
- what the data used represents 
- the solution/what you did
- major results you came up with (mention how results are measured) 

__NB:__ this final project form is much more report-like than the proposal and the checkpoint. Think in terms of writing a paper with bits of code in the middle to make the plots/tables

# Abstract (130 words)
Is it possible to design an AI algorithm that behaves similar to human players ranging from beginning to master for the simple-styled Shanghai mahjong? The data used represents training data by simulations and measures the skill expression of our agent given a starting hand. Our environment is used to guide the AI algorithm how to behave as an agent with a goal to win but with differentiating skill levels. The performance level is measured by playing alone, against other players and an oracle agents to reach the intended state of play. Though the original standard for the AI algorithm was not met, the process of designing a purposeful mahjong environment, reward prediction algorithm, functionable agent, and monte carlo algorithm allowed for discoveries of efficiencies, and reasonable simplifications for runtime/feasibility.

# Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 3 to 5 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts. 

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated. 

Mahjong is an iconic multiplayer imperfect information game that has been played for nearly 2 centuries around the world, originating in China in the mid-1800s<a name="lang"></a>[<sup>[1]</sup>](#langnote). One of the reasons it is so beloved by millions of players is because of the flexible rulesets, point system, complex strategies, and fun it brings to all players. However, it is a game involving very much stochasticity as each of the players try to find a winning set of 14 tiles. While there are certainly many mahjong AI algorithms like those used in online games such as Mahjong Soul and Microsoft’s Suphx, the goal is to create an optimized AI algorithm for Shanghai mahjong that is fair and behaves in a manner similar to humans.


One way to analyze training data is simply by using a trainer like those created by players such as Euophrys<a name="euophrys"></a>[<sup>[2]</sup>](#euophrysnote) to scrape actual gameplay from online records (such as Mahjong Soul’s database) for different levels of play; this helps for understanding different methods of playing mahjong as well as different rulesets (Mahjong Soul’s style is called jantama/riichi mahjong). On the other hand, when designing an optimal algorithm, it is important to look into Suphx and the way deep reinforcement learning is used to formulate the best strategy.<a name="li"></a>[<sup>[3]</sup>](#linote) 

The design approach was decided with techniques such as run-time policy adaptation, global reward prediction, and oracle guiding to reach the top 0.1% in terms of ranking on the Tenhou gaming service.<a name="li"></a>[<sup>[3]</sup>](#linote) Due to the nature of the game having up to 53 tiles of the total 136 tiles being hidden at any given time. It is difficult to build a standard game tree so certain techniques such as Monte-Carlo tree search and search min-max algorithms have limited applications. The way Suphx circumvents these difficulties for jantama is through policy gradient algorithms, global reward prediction of a final reward of hypothetical future rounds via learning signals, agent based training on an oracle agent with perfect information that slowly decreases access until it becomes a normal agent, and finally, a parametric Monte-Carlo policy adaptation to improve run-time.<a name="li"></a>[<sup>[3]</sup>](#linote) It is important to keep in mind that the difficulties experienced when designing an implementation for an optimal riichi mahjong AI may not be the exact same for a simplistic Shanghai mahjong ruleset, especially if the AI is more imitation/difficulty based in design. Additionally, knowing data on opposing players is not necessary to focus on the most optimal move, so simulating matches without pre-existing data is more logical and less computationally intensive.

The core framework of mahjong can be compartmentalized to four key considerations that can be used for structuring an AI: mahjong is a multi-agent four player game with no teams; mahjong is imperfect information so nothing about the hidden tiles or opponents’ tiles are directly known; not including the 13 starting tiles that are known, the initial possible states, there are nearly 10^48 possible hidden states in the set of all states, forming a hidden Markov model for each player; and finally, the cost-reward application for mahjong ranges completely differently depending on the game state, as a good hand very much depends on the hidden Markov model in order to determine possible winning hands.<a name="koyamada"></a>[<sup>[4]</sup>](#koyamadanote) This framework can be applied to both riichi and Shanghai mahjong and will serve as the fundamentals for how training data is approached and how deep reinforcement learning will be used for this particular system.

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

The general problem we are solving is winning the game of Mahjong. There are many different versions of Mahjong, which vary mainly in the system used to score winning hands, so the specific problem we are trying to solve is forming a winning hand; once this can be done, it will provide a general solution to the problem of winning any given game of Mahjong, which can be improved by further training under a given ruleset.  
  
The mechanics of a single round of Mahjong, ignoring the complicated scoring system, are relatively simple, highly standardized, and easily simulated. The mechanical properties of a given tile are wholly determined by its identity and its location, both of which can be represented as discrete variables with finite, immutable ranges, and specifying these values for a fixed set of tiles and the identity of the current player satisfies the Markov property for a round of mahjong. The rules for player actions (which are discrete and deterministic) and for terminating a round and deciding who won it are also relatively simple, and while converting them into a computer program is not the easiest of tasks, it is quite feasible. This means that it is feasible to create a perfectly accurate digital simulation of a round of Mahjong, allowing for Monte Carlo-style reinforcement learning.

##### Mechanics of Problem
For the version of Shanghai Mahjong we are using, there are 34 types of tiles: numbers 1-9 for each of three suits, and 7 unnumbered honor suits. The numbered tiles can be combined into runs of three (e.g. 3, 4, 5) of the same suit, and all tiles can be combined into sets of two- or three-of-a-kind. Each player has a hand of 13 tiles, to which one is added at the start of their turn and one removed at the end; if after adding a tile, a player's hand consists of 4 runs or sets of 3 (including a combination of runs and sets) and one set of two, that player wins. When a tile is discarded, each player who did not discard it has a chance, in turn order, to take it and form a set of three, and if none do so, the next player can take it and form a run; if any of these things happen, the set or run formed with the discarded tile cannot be broken (tiles in it cannot be discarded or used in other sets or runs). If none of these occur, or it is the first round of the game, the next player draws a tile from the wall. When a player takes a tile, it becomes their turn regardless of whether they were next in line. 

# Proposed Solution

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

We implement Mahjong playing AI by using Monte Carlo on policy method. We do it with exploring start for AI to learn many tactics on a lot of kinds of situation because Mahjong is a somewhat highly stochastic game. Even though it is, the optimal movement is highly related to the winning.

As stated in the problem statement, the objective of this game is to have a total of 4 straights or three of a kind and one pair. Each state will consist of the current hand of 13 tiles, what the opponents has tossed, and what tile has been drawn. The actions one can take is Chi, Pung, drawing a tile, discarding a tile, or nothing. The AI can only Chi a tile if it's tossed by the player before it. The AI must draw a tile on it's turn. The AI can only discard a tile if they just drew a tile, Chi, or Pung. The AI can only do nothing if an opponent tossed out a tile that can't be Chi or Pung. The values for each action will be shown below: 

| **Action**        | **Reward** |
|-------------------|------------|
| **Pung or Chi**   | +0.1       |
| **Draw, Discard, Nothing** | +0         |
| **Win**           | +1         |
| **Lose**          | -1         |

The policy we will start with is to always Pung or Chi if possible and to discard tiles that are not a pair or within two of any tile of the same suit. We set the reward not only on the winning and losing, which is the outcome, but on Pung and Chi to measure the AI performance because those are considered as good ways to become closer to winning. Furthermore we will set some rules to teach AI what kind of movement leading to the good situation and bad situation to enhance it learning optimal movement efficiently as needed. To improve the value of win rate, AI will learn as the number of simulation increase. The AI uses Monte Carlo to get better performance on many kinds of situation.

We believe this solution will work because this game contains a lot of uncertainty and variability, making a complete model near impossible. This rules out any model-based reinforcement learning and dynamic programming. Instead, by playing the game hundreds of thousands of times, we can get closer and closer to an optimal policy to this simplified version of Mahjong. This is because as the number of simulations approaches infinity, the optimal policy gets closer and closer. 

The solution will be tested against a benchmark model that functions on randomness. Every action that is avaliable for them to do will be chosen randomly and the tile it discards will also be randomly chosen. 

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

The evaluation we will use is win rate. The objective of Mahjong is to win, so having a higher winner rate means the model is doing better than it's opponents. The way win rate will be calculated will be the total number of wins a model has divided by the total number of games played. 


$$Win Rate = \Large\frac{N_{win}}{N_{total}}$$
 
Where:  
- $N_{win}$ = Number of games won  
- $N_{total}$= Total number of games played  

# Results

You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Probably you should include a learning curve to demonstrate how much better the model gets as you increase the number of trials

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Generally reinforement learning tasks may require a huge amount of training, so extensive grid search is unlikely to be possible. However expoloring a few reasonable hyper-parameters may still be possible.  Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5 

Maybe you do model selection again, but using a different kind of metric than before?  Or you compare a completely different approach/alogirhtm to the problem? Whatever, this stuff is just serving suggestions.



# Discussion

### Interpreting the result

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.


### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   


### Future work
Looking at the limitations and/or the toughest parts of the problem and/or the situations where the algorithm(s) did the worst... is there something you'd like to try to make these better.

### Ethics & Privacy

If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination.

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org

### Conclusion

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

### Future work

The biggest struggles with our work was optimizing our environment, efficiently simulating agents, and applying textbook Monte Carlo algorithms for optimization. Some steps we would have liked to try is reducing the state space from the 10^48 possible states to total possible melds or simply tiles to reduce the computational cost for both Monte Carlo and reward prediction algorithms. We additionally should have better utilized the strategy of tile encoding with 4 channels to properly calculate probabilities of a win given the current hand of the AI.

In terms of different strategies, instead of relying on simulations, we could have scraped state-action pair data from top mahjong players online or via the World Mahjong Championship records to utilize supervised learning, or focus more on run-time policy adapatation to improve optimization mid-game rather than a general strategy for training. The main roadblock for the Mahjong AI algorithm project is the computational effiency of a CPU as well as insufficient simulation time after changing strategies too far into the project. However, it is definitely possible to make use of these strategies to shorten runtime as well as greatly improve an AI agent for Shanghai mahjong based on previous precedent of Microsoft's Suphyx.<a name="li"></a>[<sup>[3]</sup>](#linote)

### Ethics & Privacy

There are two potential ethical concerns with this project. One is that if it works well enough, it could be used to cheat in high-stakes games; the other is that someone could find a way of using part of our code to do something different. Neither of these are particularly concerning - people put a lot of effort into preventing people from cheating in high-stakes gambling, and the program we’re trying to make is similar enough to other existing programs that we’re unlikely to find anything particularly new that isn’t specific to this kind of game. If we were planning on deploying it (commercially or for free), we’d also have to worry about ensuring equal access for anyone who wants it.


# Footnotes
<a name="langnote"></a>1.[^](#lang) Lang, C. (2021, May 4). What the surprising history of Mah-jongg can teach us about America. TIME. https://time.com/6045817/mahjongg-history/

<a name="euophrysnote"></a>2.[^](#euophrys) Euophrys. (n.d.). GitHub - Euophrys/Riichi-Trainer: A Riichi Mahjong trainer. GitHub. https://github.com/Euophrys/Riichi-Trainer

<a name="linote"></a>3.[^](#li) Li, J., Koyamada, S., Ye, Q., Liu, G., Wang, C., Yang, R., Zhao, L., Qin, T., Liu, T., & Hon, H. (2021, January 25). Suphx: Mastering Mahjong with Deep Reinforcement Learning - Microsoft Research. Microsoft Research. https://www.microsoft.com/en-us/research/publication/suphx-mastering-mahjong-with-deep-reinforcement-learning/

<a name="koyamadanote"></a>2.[^](#koyamada) Koyamada, S., Habara, K., Goto, N., Okano, S., Nishimori, S., & Ishii, S. (n.d.). Mjx: A framework for Mahjong AI research. 2021 IEEE Conference on Games (CoG), 504–507. https://doi.org/10.1109/cog51982.2022.9893712